In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from utils import mnist

In [2]:
train_loader, test_loader = mnist()

In [3]:
class Net(nn.Module):
    def __init__(self, log_softmax=False):
        super(Net, self).__init__()
        innerLayerShape1 = 256
        self.fc1 = nn.Linear(28*28, innerLayerShape1)
        innerLayerShape2 = 128
        self.fc2 = nn.Linear(innerLayerShape1, innerLayerShape2)
        self.fc3 = nn.Linear(innerLayerShape2, 10)
        self.log_softmax = log_softmax
        self.optim = optim.SGD(self.parameters(), lr=0.1)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        if self.log_softmax:
            x = F.log_softmax(x, dim=1)
        else:
            x = torch.log(F.softmax(x, dim=1))
        return x
    
    def loss(self, output, target, **kwargs):
        self._loss = F.nll_loss(output, target, **kwargs)
        return self._loss

In [4]:
def train(epoch, models):
    train_loss = [0]*len(models)
    train_loss_count = [0]*len(models)
    for batch_idx, (data, target) in enumerate(train_loader):
        for model in models:
            model.optim.zero_grad()
            output = model(data)
            loss = model.loss(output, target)
            loss.backward()
            model.optim.step()
            
        if batch_idx % 200 == 0:
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader))
            lossesArr = []
            for i, m in enumerate(models):
                lossModel = m._loss.item()
                train_loss[i] += lossModel
                train_loss_count[i] += 1
                lossesArr.append('{}: {:.6f}'.format(i, lossModel))
            losses = ' '.join(lossesArr)
            print(line + losses)
            
    else:
        batch_idx += 1
        line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader))
        lossesArr = []
        for i, m in enumerate(models):
            lossModel = m._loss.item()
            train_loss[i] += lossModel
            train_loss_count[i] += 1
            lossesArr.append('{}: {:.6f}'.format(i, lossModel))
        losses = ' '.join(lossesArr)
        print(line + losses)
    for i in range(len(models)):
        train_loss[i] /= train_loss_count[i]
        print('Loss: {:.4f}'.format(train_loss[i]))

In [5]:
models = [Net(), Net(True)]

In [6]:
avg_lambda = lambda l: 'Loss: {:.4f}'.format(l)
acc_lambda = lambda c, p: 'Accuracy: {}/{} ({:.0f}%)'.format(c, len(test_loader.dataset), p)
line = lambda i, l, c, p: '{}: '.format(i) + avg_lambda(l) + '\t' + acc_lambda(c, p)

def test(models):
    test_loss = [0]*len(models)
    correct = [0]*len(models)
    with torch.no_grad():
        for data, target in test_loader:
            output = [m(data) for m in models]
            for i, m in enumerate(models):
                test_loss[i] += m.loss(output[i], target, reduction='sum').item() # sum up batch loss
                pred = output[i].data.max(1, keepdim=True)[1] # get the index of the max log-probability
                correct[i] += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    for i in range(len(models)):
        test_loss[i] /= len(test_loader.dataset)
    correct_pct = [100. * c / len(test_loader.dataset) for c in correct]
    lines = '\n'.join([line(i, test_loss[i], correct[i], correct_pct[i]) for i in range(len(models))]) + '\n'
    report = 'Test set:\n' + lines
    
    print(report)

In [ ]:
for epoch in range(1, 7):
    train(epoch, models)
    test(models)

Train Epoch: 1 [0/60000 (0%)]	Losses 0: 2.297355 1: 2.293457
Train Epoch: 1 [10000/60000 (17%)]	Losses 0: 0.340604 1: 0.347098
Train Epoch: 1 [20000/60000 (33%)]	Losses 0: 0.128634 1: 0.137481
Train Epoch: 1 [30000/60000 (50%)]	Losses 0: 0.195921 1: 0.215686
Train Epoch: 1 [40000/60000 (67%)]	Losses 0: 0.049144 1: 0.058770
Train Epoch: 1 [50000/60000 (83%)]	Losses 0: 0.136484 1: 0.107077
Train Epoch: 1 [60000/60000 (100%)]	Losses 0: 0.095489 1: 0.086529
Loss: 0.4634
Loss: 0.4637
Test set:
0: Loss: 0.1197	Accuracy: 9625/10000 (96%)
1: Loss: 0.1126	Accuracy: 9641/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Losses 0: 0.061814 1: 0.039600
Train Epoch: 2 [10000/60000 (17%)]	Losses 0: 0.073433 1: 0.106139
Train Epoch: 2 [20000/60000 (33%)]	Losses 0: 0.171802 1: 0.141252
Train Epoch: 2 [30000/60000 (50%)]	Losses 0: 0.036942 1: 0.044467
Train Epoch: 2 [40000/60000 (67%)]	Losses 0: 0.058888 1: 0.056256
Train Epoch: 2 [50000/60000 (83%)]	Losses 0: 0.066679 1: 0.044743
Train Epoch: 2 [60000/60000 